# Combining Guardrails Contextual Grounding and Knowledge Bases
This notebook demonstrates how to combine Guardrails for Amazon Bedrock contextual grounding filter with the Amazon Bedrock Knowledge Bases. By doing so, we can ensure that the model's responses are factually grounded and aligned with the information stored in the Knowledge Base.

In the notebook we will be using the [D&D Systems Reference Document (SRD)](https://www.dndbeyond.com/resources/1781-systems-reference-document-srd)(CC-BY-4.0) to store it into the Knowledge Base.  

What we are going to create in this notebook:
1. **Import libraries:** We will load the needed libraries to make the notebook work correctly. 
2. **Create Knowledge Base:** We are going to store the D&D Systems Reference Document in a managed knowledge base in Amazon Bedrock. 
3. **Configure Guardrail:** We are going to configure our guardrail with the contextual grounding filter thresholds.
4. **Test the contextual grounding:** We are going to retrieve context from the KB and pass it to a LLM with a query and evaluate hallucinations. 
5. **Delete resources:** To save in costs, we are going to delete all the resources created. 

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure to enable `Anthropic Claude 3 Sonnet`and `Titan Embedding Text V2`  model access in Amazon Bedrock Console, as the notebook will use these models.
</div>

## 1. Import libraries

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


This code is part of the setup and used to :
- Add the parent directory to the python system path
- Imports a custom module (BedrockStructuredKnowledgeBase) from `utils` necessary for later executions

In [2]:
import os
import sys
import time
import boto3
import logging
import pprint
import json
from pathlib import Path
import requests
import uuid

current_path = Path().resolve()
current_path = current_path.parent

if str(current_path) not in sys.path:
    sys.path.append(str(current_path))

# Print sys.path to verify
print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase
from utils.knowledge_base_operators import print_results,print_results_with_guardrail

['/opt/conda/lib/python312.zip', '/opt/conda/lib/python3.12', '/opt/conda/lib/python3.12/lib-dynload', '', '/opt/conda/lib/python3.12/site-packages', '/mnt/custom-file-systems/efs/fs-00372d464aa72992f_fsap-0712cfbff4946f620/rag-workshop-amazon-bedrock-knowledge-bases']


In [3]:
session = boto3.session.Session()
region = session.region_name
unique_id = str(uuid.uuid4())[:4]
s3_client = boto3.client("s3",region_name=region)
bedrock = boto3.client("bedrock",region_name=region)
bedrock_runtime = boto3.client("bedrock-runtime",region_name=region)
bedrock_agent_client = boto3.client("bedrock-agent",region_name=region)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime",region_name=region)

## 2. Create Knowledge Base

### 2.1 Download the dataset

In [4]:
url = "https://media.wizards.com/2023/downloads/dnd/SRD_CC_v5.1.pdf"
file_name = "kb_documents/SRD_CC_v5.1.pdf"
os.makedirs("kb_documents", exist_ok=True)
response = requests.get(url)
with open(file_name, "wb") as file:
    file.write(response.content)
print(f"File '{file_name}' has been downloaded.")

File 'kb_documents/SRD_CC_v5.1.pdf' has been downloaded.


### 2.1 Creating Knowledge Base

We will now going to create a Knowledge Base and its requirements including:
- [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/) for the vector database
- [AWS IAM](https://aws.amazon.com/iam/) roles and permissions
- [Amazon S3](https://aws.amazon.com/s3/) bucket to store the knowledge base documents

To create the knowledge base and its dependencies, we will use the `BedrockKnowledgeBase` support class, available in this folder. It allows you to create a new knowledge base, ingest documents to the knowledge base data source and delete the resources after you are done working with this lab

In [5]:
knowledge_base_name = "{}-cgdemo".format(unique_id)
knowledge_base_description = "Knowledge Base containing d&d Guide"
bucket_name = "{}-cgdemo-bucket".format(unique_id)

#### Define data source

Define the data source to be connevted to your knowledge base. YWith Amazon Bedrock Knowledge Bases, you have multiple options to choose from various data sources (S3, Sharepoint, Salefforce, Confluence).  For this notebook, we'll be using S3 bucket as data source.

In [6]:
data_sources=[{"type": "S3", "bucket_name": bucket_name}]

In [7]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=knowledge_base_name,
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{unique_id}-f'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['b457-cgdemo-bucket']
buckets_to_check:  ['b457-cgdemo-bucket']
Creating bucket b457-cgdemo-bucket
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_b457-f) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '317',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 04 Dec 2025 05:52:29 '
                                                 'GMT',
                                         'x-amzn-requestid': '7849a40a-cdcd-446d-9f0b-53361ad110ff'},
                        'HTTPStatusCode': 200,
                        'RequestId': '7849a40a-cdcd-446d

We now upload the knowledge base documents to S3

In [8]:
def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".pdf"):
                file_to_upload = os.path.join(root, file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)

upload_directory("kb_documents", bucket_name)

uploading file kb_documents/SRD_CC_v5.1.pdf to b457-cgdemo-bucket


And ingest the documents to the knowledge base

In [9]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'HYDPLF8ZQQ',
  'ingestionJobId': 'ZYFDFXECDK',
  'knowledgeBaseId': 'M8ECYLHPCH',
  'startedAt': datetime.datetime(2025, 12, 4, 5, 55, 33, 983286, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 12, 4, 5, 56, 56, 57662, tzinfo=tzlocal())}
........................................

In [10]:
kb_id = knowledge_base.get_knowledge_base_id()

'M8ECYLHPCH'


### 2.2 Testing Knowledge Base
Let's now test that the created knowledge base works as expected. To do so, we first retrieve the knowledge base id. 

Next we can use the [`RetrieveAndGenerate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 
API from boto3 to retrieve the context for the question from the knowledge base and generate the final response

In [11]:
model_id="anthropic.claude-3-sonnet-20240229-v1:0"
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "What should I know about elves?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_id,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

Elves are a race with several unique traits:

- They do not need to sleep, but instead enter a meditative trance for 4 hours per day. During this trance, they can dream and gain the same benefit as 8 hours of sleep for humans.
- They have superior vision in dark and dim conditions, allowing them to see in dim light as if it were bright light and in darkness as if it were dim light (though they cannot discern color in complete darkness).
- They have proficiency in the Perception skill due to their keen senses.
- They have advantage on saving throws against being charmed, and magic cannot put them to sleep. There are two main types of elves:

1) High elves who have a keen mind and mastery of at least basic magic. Some high elves are haughty and reclusive, believing themselves superior to non-elves and other elves.

2) A more common and friendly type of elf that is often encountered among humans and other races.

All elves have increased Dexterity, can live over 750 years, and have profic

## 3. Configure Guardrail for Amazon Bedrock

Now we have the Knowledge Base created, configured and synced with our documents, let's go and create our Guardrail for Amazon Bedrock. 

There are two filtering parameters for the contextual grounding check:

- **Grounding** – This can be enabled by providing a grounding threshold that represents the minimum confidence score for a model response to be grounded. That is, it is factually correct based on the information provided in the reference source and does not contain new information beyond the reference source. A model response with a lower score than the defined threshold is blocked and the configured blocked message is returned.

- **Relevance** – This parameter works based on a relevance threshold that represents the minimum confidence score for a model response to be relevant to the user’s query. Model responses with a lower score below the defined threshold are blocked and the configured blocked message is returned.

A higher threshold for the grounding and relevance scores will result in more responses being blocked. Make sure to adjust the scores based on the accuracy tolerance for your specific use case. For example, a customer-facing application in the finance domain may need a high threshold due to lower tolerance for inaccurate content.

In [12]:
response = bedrock.create_guardrail(
    name="contextual-grounding-guardrail-{}".format(unique_id),
    description="D&D Guardrail",
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.5
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.8
            },
        ]
    },
    blockedInputMessaging="Sorry, I can not respond to this.",
    blockedOutputsMessaging="Sorry, I can not respond to this.",
)
guardrailId = response["guardrailId"]
print("The guardrail id is",response["guardrailId"])

The guardrail id is bjf53pdab5sw


## 4. Test the contextual grounding capability
Now we have set up the Knowledge Base and Guardrail let's test them together.

In this section we will first retrieve the KB results and then pass it on to the Converse API which has the Guardrail integrated.

In [13]:
def invoke_kb(kb_query):
    kb_response = bedrock_agent_runtime_client.retrieve(
        knowledgeBaseId=kb_id,
        retrievalConfiguration={
            'vectorSearchConfiguration': {
                'numberOfResults': 2,
            }
        },
        retrievalQuery={
            'text': kb_query
        }
    )
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
 
    inference_config = {"temperature": 0.1}

    # The message for the model and the content that you want the guardrail to assess.
    messages = [
        {
            "role": "user",
            "content": [
                {"text": str(kb_response)},
                {"text": kb_query}
            ]
        }
    ]
    response = bedrock_runtime.converse(modelId=model_id,messages=messages, inferenceConfig=inference_config)
    print("""
    ================================
    Invoke KB without Guardrails
    ================================
    """)
    print_results(kb_response, response)


def invoke_kb_with_guardrail(kb_query):
    kb_response = bedrock_agent_runtime_client.retrieve(
        knowledgeBaseId=kb_id,
        retrievalConfiguration={
            'vectorSearchConfiguration': {
                'numberOfResults': 2,
            }
        },
        retrievalQuery={
            'text': kb_query
        }
    )
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
    inference_config = {"temperature": 0.1}
    guardrail_config = {
        "guardrailIdentifier": guardrailId,
        "guardrailVersion": "DRAFT",
        "trace": "enabled"
    }

    # The message for the model and the content that you want the guardrail to assess.
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "guardContent": {
                        "text": {
                            "text": str(kb_response),
                            "qualifiers": ["grounding_source"],
                        }
                    }
                },
                {
                    "guardContent": {
                        "text": {
                            "text": kb_query,
                            "qualifiers": ["query"],
                        }
                    }
                },
            ],
        }
    ]
    response = bedrock_runtime.converse(modelId=model_id,messages=messages,guardrailConfig=guardrail_config, inferenceConfig=inference_config,)
    print("""
    ================================
    Invoke KB with Guardrails
    ================================
    """)
    print_results_with_guardrail(kb_response, response)


In [14]:
kb_query = "What are High Elves?"
invoke_kb(kb_query)
invoke_kb_with_guardrail(kb_query)


    Invoke KB without Guardrails
    
Knowledge Base retrieval results:

Chunk 1:
Elvish is fluid, with subtle intonations and intricate grammar. Elven literature is rich and varied, and their songs and poems are famous among other races. Many bards learn their language so they can add Elvish ballads to their repertoires. High Elf As a high elf, you have a keen mind and a mastery of at least the basics of magic. In many fantasy gaming worlds, there are two kinds of high elves. One type is haughty and reclusive, believing themselves to be superior to non-­‐‑elves and even other elves. 

Chunk 2:
Your hit point maximum increases by 1, and it increases by 1 every time you gain a level. Elf Elf Traits Your elf character has a variety of natural abilities, the result of thousands of years of elven refinement. Ability Score Increase. Your Dexterity score increases by 2. Age. Although elves reach physical maturity at about the same age as humans, the elven understanding of adulthood goes bey

In [15]:
kb_query = "Where should the elves go if they arrive in Paris?"
invoke_kb(kb_query)
invoke_kb_with_guardrail(kb_query)


    Invoke KB without Guardrails
    
Knowledge Base retrieval results:

Chunk 1:
Trance. Elves don’t need to sleep. Instead, they meditate deeply, remaining semiconscious, for 4 hours a day. (The Common word for such meditation is “trance.”) While meditating, you can dream after a fashion; such dreams are actually mental exercises that have become reflexive through years of practice. After resting in this way, you gain the same benefit that a human does from 8 hours of sleep. Languages. You can speak, read, and write Common and Elvish. Elvish is fluid, with subtle intonations and intricate grammar. 

Chunk 2:
Darkvision. Accustomed to twilit forests and the night sky, you have superior vision in dark and dim conditions. You can see in dim light within 60 feet of you as if it were bright light, and in darkness as if it were dim light. You can’t discern color in darkness, only shades of gray. Keen Senses. You have proficiency in the Perception skill. Fey Ancestry. You have advantage on

## 5. Delete resources
Let's delete all the resources to avoid unnecessary costs. 

In [16]:
## Delete the Knowledge Base
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
## Delete the Guardrail
bedrock.delete_guardrail(guardrailIdentifier = guardrailId)

Deleted data source HYDPLF8ZQQ
======== Knowledge base and all data sources deleted =========
Found bucket b457-cgdemo-bucket
Deleted all objects in bucket b457-cgdemo-bucket
Deleted bucket b457-cgdemo-bucket
======== S3 bucket deletion process completed =========
Found role AmazonBedrockExecutionRoleForKnowledgeBase_b457-f
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_b457-f========
 [{'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_b457-f', 'PolicyArn': 'arn:aws:iam::125929885348:policy/AmazonBedrockOSSPolicyForKnowledgeBase_b457-f'}, {'PolicyName': 'AmazonBedrockCloudWatchPolicyForKnowledgeBase_b457-f', 'PolicyArn': 'arn:aws:iam::125929885348:policy/AmazonBedrockCloudWatchPolicyForKnowledgeBase_b457-f'}, {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_b457-f', 'PolicyArn': 'arn:aws:iam::125929885348:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_b457-f'}, {'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_b457-f',

{'ResponseMetadata': {'RequestId': 'be4be362-00b5-457b-88b8-bd70c00c0571',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 04 Dec 2025 05:58:05 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'be4be362-00b5-457b-88b8-bd70c00c0571'},
  'RetryAttempts': 0}}